In [1]:
import yaml
import numpy as np
import os
import soundfile as sf
from scipy.signal import stft
from tqdm import tqdm
import torch
# If you have a GPU, put the data on the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

directory = "../Data/slakh2100_flac_redux/train"
listdir = os.listdir(directory)
listdir.sort()
format = '.flac'
savedir = 'data/train'
sample_freq = 44100
freq_amount = 129
max_size = 200
listdir = listdir[:max_size]

In [2]:
for i, tr in enumerate(tqdm(listdir)):
    tr_dicts_2 = {}
    tr_path = os.path.join(directory, tr)
    with open(os.path.join(tr_path, "metadata.yaml")) as meta:
        metadata = yaml.load(meta, Loader=yaml.Loader)
    # add Mix to the dictionary
    f, t, Zxx = stft(sf.read(os.path.join(directory,tr, 'mix'+format))[0], fs=sample_freq, nperseg=freq_amount*2-2)
    tr_dicts_2.update({'mix': torch.view_as_real_copy(torch.tensor(Zxx, dtype=torch.complex128, device=device))})
    file_inst = []
    for stem in metadata['stems'].keys():
        file_inst.append([stem + format, metadata['stems'][stem]['inst_class']])
    file_inst = np.array(file_inst)
    file_inst = file_inst[[file_inst[:,0][i] in os.listdir(tr_path + '/stems') for i in range(len(file_inst))]]
    # combine all stems from the same instrument and track using soundfile
    for inst in np.unique(file_inst[:,1]):
        inst_files = file_inst[file_inst[:,1] == inst][:,0]
        inst_files = [os.path.join(tr_path, 'stems', inst_file) for inst_file in inst_files]
        inst_data = np.array([sf.read(inst_file)[0] for inst_file in inst_files])
        inst_data = np.sum(inst_data, axis=0)
        # compute the STFT of the combined data
        if inst in ['mix','Bass', 'Guitar', 'Drums', 'Piano']:
            f, t, Zxx = stft(inst_data, fs=sample_freq, nperseg=freq_amount*2-2)
            tr_dicts_2.update({inst: torch.view_as_real_copy(torch.tensor(Zxx, dtype=torch.complex128, device=device))})
    # save the dictionary as a .pt file
    torch.save(tr_dicts_2, os.path.join(savedir, tr + '.pt'))
    torch.cuda.empty_cache()

100%|██████████| 200/200 [12:24<00:00,  3.72s/it]
